In [1]:
from my_packages import *
from appgeopy import *
warnings.filterwarnings("ignore")

**Extract PSC within segments**

Input file:
- segment shapefiles
    - segment file required column: C_Length (cumulative length) & Name (Seg_{!C_Length!})
- ps point data shapefile

Implemented fuction:
- `PSC_Within_Profile_Segment()`

In [2]:
def export_parameter_record(savefolder, params_dict):

    import os

    for i in range(1, 1000):
        savepath = os.path.join(savefolder, "Param_Record_{}.txt".format(str(i).zfill(3)))
        if os.path.exists(savepath):
            continue
        else:
            break

    with open(savepath, "w+") as file:
        for key, value in params_dict.items():
            file.write("{} : {}".format(key, value))
            file.write("\n\n")
    pop_up_string = "Saved at {}".format(savefolder)
    print(pop_up_string)


def open_geodata(filepath):
    import sys

    import geopandas as gpd

    geodata_type = type(gpd.GeoDataFrame())

    if type(filepath) == geodata_type:
        geodata = filepath
    elif type(filepath) == str:
        extension = filepath.split(".")[-1]
        if extension == "pkl":
            geodata = pd.read_pickle(filepath, compression="zip")
        elif extension == "shp":
            geodata = gpd.read_file(filepath)
        else:
            sys.exit(
                "The datatype of PSC_filepath is invalid.\
            \nExpect {} or {}, not {}".format(
                    type("1"), geodata_type, type(PSC_filepath)
                )
            )
    else:
        sys.exit("{} invalid filetype".format(type(filepath)))
    return geodata


def PSC_Within_Profile_Segment(psc_filepath, segment_filepath, distance=200):
    # ---------------------------------------------
    import warnings

    warnings.filterwarnings("ignore")

    import os
    import sys

    import geopandas as gpd
    import pandas as pd
    from tqdm import tqdm, trange

    # ---------------------------------------------
    radius = distance

    # đọc shapefile của PS candidates

    PSC_data = open_geodata(psc_filepath)

    # đọc shapefile của profile segment
    segment_data = open_geodata(segment_filepath)
    segment_data_length = len(segment_data)

    # một dataframe trống để chứa thông tin các điểm bên trong buffer
    empty_table = pd.DataFrame(data=None)

    for i, _temp in zip(range(segment_data_length), trange(segment_data_length)):
        try:
            # profile_segment = segment_data.loc[i, :]
            # profile_segment_buffer = profile_segment.geometry.buffer(radius)

            profile_segment_buffer = segment_data.loc[i, :].geometry

            psc_within_buffer = PSC_data.geometry.within(profile_segment_buffer)
            selected_PSC_table = PSC_data[psc_within_buffer]

            segment_name = segment_data.loc[i, "Name"]
            selected_PSC_table["Segment"] = segment_name

            empty_table = pd.concat([empty_table, selected_PSC_table])
        except Exception as e:
            sys.exit("{} at location {}".format(e, i))

    empty_table.reset_index(inplace=True, drop=True)

    new_geospatial_datatable = gpd.GeoDataFrame(
        data=empty_table, geometry=empty_table.geometry, crs=empty_table.crs
    )

    return new_geospatial_datatable

#### Input Data

In [3]:
geodata_filepath = r"D:\003_PRESENTATIONS\20_TSX_Sentinel_Report\calibrated\calibrated_S1A_tsERA5_avgvelocity.pkl"
segment_shp_filepath = (
    r"E:\002_ARCGIS_WORK\ManuscriptArcGIS\HSR_Shapefiles\BUFFER\BUFFER_HSR_Segment_200m_TWD97.shp"
)

savefolder = r"D:\003_PRESENTATIONS\20_TSX_Sentinel_Report\hsr_section"
extension_option = "pkl"

params_dict = {
    "geodata_filepath": geodata_filepath,
    "segment_shp_filepath": segment_shp_filepath,
    "savefolder": savefolder,
}

In [4]:
input_geodata = pd.read_pickle(geodata_filepath, compression='zip')
input_geodata = geospatial.convert_to_geodata(df=input_geodata, xcoord_col="POINT_X", ycoord_col="POINT_Y", crs_epsg="EPSG:3826")
input_geodata.head(5)

,OBJECTID,CODE,HEIGHT,H_STDEV,VEL,V_STDEV,COHERENCE,EFF_AREA,POINT_X,POINT_Y,PointKey,velocity_mm_per_day,velocity_mm_per_year,geometry
0,1,00000001,28.020000,0.0,2.240000,7.28,0.712,1,178147.1514,2.630709e+06,X178147Y2630708,-0.206489,-75.420055,POINT (178147.151 2630708.793)
1,2,00000002,27.969999,0.0,8.710000,7.63,0.714,1,178159.5813,2.630706e+06,X178159Y2630706,-0.151533,-55.347293,POINT (178159.581 2630706.407)
2,3,00000003,27.570000,0.0,11.590000,13.49,0.689,1,178172.0111,2.630704e+06,X178172Y2630704,0.224200,81.889164,POINT (178172.011 2630704.022)
3,4,00000004,29.040001,0.0,-47.709999,7.88,0.857,1,178293.8738,2.630659e+06,X178293Y2630659,-0.349807,-127.767106,POINT (178293.874 2630659.268)
4,6,00000006,27.959999,0.0,1.940000,13.55,0.846,1,178310.2398,2.630667e+06,X178310Y2630666,-0.307696,-112.386114,POINT (178310.240 2630666.581)


#### Data Processing

In [5]:
psc_data = PSC_Within_Profile_Segment(
    psc_filepath=input_geodata, segment_filepath=segment_shp_filepath
)

100%|███████████████████████████████████████████████████████████████████████████████▉| 978/979 [00:45<00:00, 21.70it/s]


In [6]:
psc_data

,OBJECTID,CODE,HEIGHT,H_STDEV,VEL,V_STDEV,COHERENCE,EFF_AREA,POINT_X,POINT_Y,PointKey,velocity_mm_per_day,velocity_mm_per_year,geometry,Segment
0,30460,000076fc,42.410000,0.0,9.580000,5.02,0.999,1,191442.0792,2.628111e+06,X191442Y2628111,-0.177695,-64.903113,POINT (191442.079 2628111.332),Seg_28500
1,28177,00006e11,42.520001,0.0,16.709999,5.85,1.000,1,191065.9593,2.628185e+06,X191065Y2628184,-0.158085,-57.740586,POINT (191065.959 2628184.677),Seg_28550
2,28178,00006e12,43.020001,0.0,17.820000,4.65,0.998,1,191077.6163,2.628183e+06,X191077Y2628182,-0.140757,-51.411453,POINT (191077.616 2628182.517),Seg_28550
3,28179,00006e13,43.880001,0.0,18.740000,4.80,0.999,1,191087.7188,2.628181e+06,X191087Y2628180,-0.134751,-49.217728,POINT (191087.719 2628180.575),Seg_28550
4,28569,00006f99,44.709999,0.0,22.000000,5.00,0.997,1,191098.5981,2.628178e+06,X191098Y2628178,-0.149761,-54.700289,POINT (191098.598 2628178.419),Seg_28550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12365,650127,0009eb8f,33.290001,0.0,14.030000,7.28,0.998,1,182027.3779,2.610225e+06,X182027Y2610225,-0.146238,-53.413503,POINT (182027.378 2610225.209),Seg_49000
12366,650128,0009eb90,33.520001,0.0,-6.140000,7.91,0.999,1,182039.0473,2.610223e+06,X182039Y2610222,-0.172195,-62.894402,POINT (182039.047 2610222.831),Seg_49000
12367,650129,0009eb91,26.790001,0.0,-17.139999,6.56,1.000,1,182120.8331,2.610228e+06,X182120Y2610227,-0.161585,-59.018796,POINT (182120.833 2610227.731),Seg_49000
12368,650361,0009ec79,30.370001,0.0,-3.410000,7.44,0.828,1,181994.6055,2.610211e+06,X181994Y2610210,-0.155154,-56.669976,POINT (181994.605 2610210.786),Seg_49000


#### Save Data

In [7]:
pre_columns = psc_data.columns.tolist()
new_columns = pre_columns[:-2] + [pre_columns[-1]] + ["geometry"]
output_data = psc_data[new_columns].copy()

In [8]:
output_data.head(5)

,OBJECTID,CODE,HEIGHT,H_STDEV,VEL,V_STDEV,COHERENCE,EFF_AREA,POINT_X,POINT_Y,PointKey,velocity_mm_per_day,velocity_mm_per_year,Segment,geometry
0,30460,000076fc,42.410000,0.0,9.580000,5.02,0.999,1,191442.0792,2.628111e+06,X191442Y2628111,-0.177695,-64.903113,Seg_28500,POINT (191442.079 2628111.332)
1,28177,00006e11,42.520001,0.0,16.709999,5.85,1.000,1,191065.9593,2.628185e+06,X191065Y2628184,-0.158085,-57.740586,Seg_28550,POINT (191065.959 2628184.677)
2,28178,00006e12,43.020001,0.0,17.820000,4.65,0.998,1,191077.6163,2.628183e+06,X191077Y2628182,-0.140757,-51.411453,Seg_28550,POINT (191077.616 2628182.517)
3,28179,00006e13,43.880001,0.0,18.740000,4.80,0.999,1,191087.7188,2.628181e+06,X191087Y2628180,-0.134751,-49.217728,Seg_28550,POINT (191087.719 2628180.575)
4,28569,00006f99,44.709999,0.0,22.000000,5.00,0.997,1,191098.5981,2.628178e+06,X191098Y2628178,-0.149761,-54.700289,Seg_28550,POINT (191098.598 2628178.419)


In [9]:
base = os.path.basename(segment_shp_filepath).split(".")[0]

if extension_option == "pkl":
    savename = "Output_" + base + ".pkl"
    savepath = os.path.join(savefolder, savename)
    output_data.to_pickle(savepath, compression="zip")
elif extension_option == "shp":
    savename = "Output_" + base + ".shp"
    savepath = os.path.join(savefolder, savename)
    # output_data.to_file(savepath)
else:
    raise ("Invalid filetype!")

export_parameter_record(savefolder=savefolder, params_dict=params_dict)

Saved at D:\003_PRESENTATIONS\20_TSX_Sentinel_Report\hsr_section
